<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Singapore Housing Data and Kaggle Challenge

## Problem Statement

When it comes to property prices in Singapore, the on-ground sentiments we often hear are — "so expensive", "so far", "is it worth the price?"

Yet, there still seems to be a preference amongst some to stay in the central region due to convenience and general accessibility. In our study, we examine how the price of resale HDB flats, particularly in the central region, are influenced by various factors like floor area, age of HDB flat, maximum floor level, proximity to amenities and public transport connectivity.

Ultimately, we aim to address the following question: **"Are resale prices of central region HDBs influenced primarily by its location?"**

In doing so, we hope to empower our target audience to make more calculated and informed decisions on housing whether it's young couples buying their first flats, or older families looking to sell their flats

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
import warnings
import pickle
warnings.filterwarnings('ignore')
plt.style.use('seaborn-pastel')
pd.set_option('display.max_rows', 200)

## Data Import and Cleaning

### Cleaning the Data

In [2]:
train = pd.read_csv('datasets/train.csv')

test = pd.read_csv('datasets/test.csv')


#### Print the first 5 rows of each dataframe

In [3]:
train.head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,...,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,...,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,...,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,...,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,...,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


In [4]:
test.head()


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,114982,2012-11,YISHUN,4 ROOM,173,YISHUN AVE 7,07 TO 09,84.0,Simplified,1987,...,92,0,1.433681,103.832924,156.322353,Ahmad Ibrahim Secondary School,218,0,1.436235,103.829987
1,95653,2019-08,JURONG WEST,5 ROOM,986C,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,2008,...,45,0,1.339244,103.698896,739.371688,Jurong West Secondary School,199,0,1.335256,103.702098
2,40303,2013-10,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,36,0,1.371893,103.851811,305.071191,Anderson Secondary School,245,0,1.374242,103.851430
3,109506,2017-10,WOODLANDS,4 ROOM,29,MARSILING DR,01 TO 03,97.0,New Generation,1979,...,54,0,1.434423,103.773698,433.454591,Woodlands Secondary School,188,0,1.439183,103.774499
4,100149,2016-08,BUKIT BATOK,4 ROOM,170,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,1985,...,40,0,1.349195,103.741000,217.295361,Bukit Batok Secondary School,223,0,1.348351,103.740873


In [5]:
# Check data types (train)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 78 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         150634 non-null  int64  
 1   Tranc_YearMonth            150634 non-null  object 
 2   town                       150634 non-null  object 
 3   flat_type                  150634 non-null  object 
 4   block                      150634 non-null  object 
 5   street_name                150634 non-null  object 
 6   storey_range               150634 non-null  object 
 7   floor_area_sqm             150634 non-null  float64
 8   flat_model                 150634 non-null  object 
 9   lease_commence_date        150634 non-null  int64  
 10  resale_price               150634 non-null  float64
 11  Tranc_Year                 150634 non-null  int64  
 12  Tranc_Month                150634 non-null  int64  
 13  mid_storey                 15

In [6]:
# Check data types (test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 77 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         16737 non-null  int64  
 1   Tranc_YearMonth            16737 non-null  object 
 2   town                       16737 non-null  object 
 3   flat_type                  16737 non-null  object 
 4   block                      16737 non-null  object 
 5   street_name                16737 non-null  object 
 6   storey_range               16737 non-null  object 
 7   floor_area_sqm             16737 non-null  float64
 8   flat_model                 16737 non-null  object 
 9   lease_commence_date        16737 non-null  int64  
 10  Tranc_Year                 16737 non-null  int64  
 11  Tranc_Month                16737 non-null  int64  
 12  mid_storey                 16737 non-null  int64  
 13  lower                      16737 non-null  int

In [7]:
#checking for duplicates (train)
train.duplicated().value_counts()

False    150634
dtype: int64

In [8]:
#checking for duplicates (test)
test.duplicated().value_counts()

False    16737
dtype: int64

#### Convert Y/N to Binary/Boolean

In [9]:
# List of columns to convert from 'Y'/'N' to 1/0 (train dataset)
columns_to_convert = ['residential', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion']

# Loop through each column and apply the mapping
for column in columns_to_convert:
    train[column] = train[column].map({'Y': 1, 'N': 0})

# Optional: Display the first few rows to verify the conversion
print(train[columns_to_convert].head())


   residential  commercial  market_hawker  multistorey_carpark  \
0            1           0              0                    0   
1            1           0              0                    0   
2            1           0              0                    0   
3            1           1              0                    0   
4            1           0              0                    0   

   precinct_pavilion  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  


In [10]:
# List of columns to convert from 'Y'/'N' to 1/0 (test dataset)
columns_to_convert = ['residential', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion']

# Loop through each column and apply the mapping
for column in columns_to_convert:
    test[column] = test[column].map({'Y': 1, 'N': 0})

# Optional: Display the first few rows to verify the conversion
print(test[columns_to_convert].head())

   residential  commercial  market_hawker  multistorey_carpark  \
0            1           1              0                    0   
1            1           0              0                    0   
2            1           0              0                    0   
3            1           0              0                    0   
4            1           0              0                    0   

   precinct_pavilion  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  


In [11]:
train['planning_area'].unique()

array(['Kallang', 'Bishan', 'Bukit Batok', 'Yishun', 'Geylang', 'Hougang',
       'Bedok', 'Sengkang', 'Tampines', 'Serangoon', 'Bukit Merah',
       'Bukit Panjang', 'Woodlands', 'Jurong West', 'Toa Payoh',
       'Choa Chu Kang', 'Sembawang', 'Novena', 'Ang Mo Kio', 'Pasir Ris',
       'Clementi', 'Punggol', 'Jurong East', 'Rochor', 'Queenstown',
       'Bukit Timah', 'Outram', 'Tanglin', 'Marine Parade',
       'Western Water Catchment', 'Downtown Core', 'Changi'], dtype=object)

In [12]:
train['town'].unique()

array(['KALLANG/WHAMPOA', 'BISHAN', 'BUKIT BATOK', 'YISHUN', 'GEYLANG',
       'HOUGANG', 'BEDOK', 'SENGKANG', 'TAMPINES', 'SERANGOON',
       'BUKIT MERAH', 'BUKIT PANJANG', 'WOODLANDS', 'JURONG WEST',
       'TOA PAYOH', 'CHOA CHU KANG', 'SEMBAWANG', 'ANG MO KIO',
       'PASIR RIS', 'CLEMENTI', 'PUNGGOL', 'JURONG EAST', 'CENTRAL AREA',
       'QUEENSTOWN', 'BUKIT TIMAH', 'MARINE PARADE'], dtype=object)

In [13]:
# List of values to binary/boolean
columns_to_convert = ['residential', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion']

# Loop through each column and apply the mapping
for column in columns_to_convert:
    test[column] = test[column].map({'Y': 1, 'N': 0})

# Optional: Display the first few rows to verify the conversion
print(test[columns_to_convert].head())

   residential  commercial  market_hawker  multistorey_carpark  \
0          NaN         NaN            NaN                  NaN   
1          NaN         NaN            NaN                  NaN   
2          NaN         NaN            NaN                  NaN   
3          NaN         NaN            NaN                  NaN   
4          NaN         NaN            NaN                  NaN   

   precinct_pavilion  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 78 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         150634 non-null  int64  
 1   Tranc_YearMonth            150634 non-null  object 
 2   town                       150634 non-null  object 
 3   flat_type                  150634 non-null  object 
 4   block                      150634 non-null  object 
 5   street_name                150634 non-null  object 
 6   storey_range               150634 non-null  object 
 7   floor_area_sqm             150634 non-null  float64
 8   flat_model                 150634 non-null  object 
 9   lease_commence_date        150634 non-null  int64  
 10  resale_price               150634 non-null  float64
 11  Tranc_Year                 150634 non-null  int64  
 12  Tranc_Month                150634 non-null  int64  
 13  mid_storey                 15

#### Change dtypes

Change dates from obj to datetime64[ns]

In [15]:
# Transacted month (train)
train['Tranc_YearMonth'] = pd.to_datetime(train['Tranc_YearMonth'])

train['Tranc_YearMonth'] = train['Tranc_YearMonth'].dt.strftime('%Y-%m')

In [16]:
# Transacted month (test)
test['Tranc_YearMonth'] = pd.to_datetime(test['Tranc_YearMonth'])

test['Tranc_YearMonth'] = test['Tranc_YearMonth'].dt.strftime('%Y-%m')

#### Dropping the unnecessary columns [Reason: redundancy due to duplicate data/metrics, redundant data]

In [17]:
# Drop the latitude and longtitude columns (train)
train.drop(['Latitude', 'Longitude', 'mrt_latitude', 'mrt_longitude', 'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_longitude', 'pri_sch_latitude', 'sec_sch_longitude', 'sec_sch_latitude'], axis = 1, inplace = True)

# Drop the unique values that has no contributory value
train.drop(['block', 'street_name', 'address', 'postal', 'bus_stop_name'], axis = 1, inplace = True)

# Drop duplicates (in measurement or metric)
train.drop(['Tranc_YearMonth', 'Tranc_Month', 'floor_area_sqm', 'full_flat_type', 'lease_commence_date', 'year_completed', 'price_per_sqft'], axis = 1, inplace = True)

# Drop 'precinct_pavilion', because there's no adjacent feature that can support any analysis on this
train.drop(['precinct_pavilion'], axis = 1, inplace = True)

# Drop 'residential' and 'commercial', as we're assuming that it's all residential
train.drop(['residential', 'commercial'], axis = 1, inplace = True)

# Drop rental, we're not looking into rental, more of selling-buying
train.drop(['1room_rental', '2room_rental', '3room_rental', 'other_room_rental'], axis = 1, inplace = True)

# Drop irrelevant education/academic features
train.drop(['pri_sch_affiliation'], axis = 1, inplace = True)

train.head()


,id,town,flat_type,storey_range,flat_model,resale_price,Tranc_Year,mid_storey,lower,upper,...,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_name,vacancy,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation
0,88471,KALLANG/WHAMPOA,4 ROOM,10 TO 12,Model A,680000.0,2016,11,10,12,...,0,0,29.427395,1138.633422,Geylang Methodist School,78,1138.633422,Geylang Methodist School,224,0
1,122598,BISHAN,5 ROOM,07 TO 09,Improved,665000.0,2012,8,7,9,...,1,1,58.207761,415.607357,Kuo Chuan Presbyterian Primary School,45,447.894399,Kuo Chuan Presbyterian Secondary School,232,0
2,170897,BUKIT BATOK,EXECUTIVE,13 TO 15,Apartment,838000.0,2013,14,13,15,...,1,0,214.747860,498.849039,Keming Primary School,39,180.074558,Yusof Ishak Secondary School,188,0
3,86070,BISHAN,4 ROOM,01 TO 05,Model A,550000.0,2012,3,1,5,...,1,1,43.396521,389.515528,Catholic High School,20,389.515528,Catholic High School,253,1
4,153632,YISHUN,4 ROOM,01 TO 03,Simplified,298000.0,2017,2,1,3,...,0,0,129.422752,401.200584,Naval Base Primary School,74,312.025435,Orchid Park Secondary School,208,0


In [18]:
train.duplicated().value_counts()

False    150634
dtype: int64

In [19]:
test.duplicated().value_counts()

False    16737
dtype: int64

In [20]:
# Drop the latitude and longtitude columns (test)
test.drop(['Latitude', 'Longitude', 'mrt_latitude', 'mrt_longitude', 'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_longitude', 'pri_sch_latitude', 'sec_sch_longitude', 'sec_sch_latitude'], axis = 1, inplace = True)

# Drop the unique values that has no contributory value
test.drop(['block', 'street_name', 'address', 'postal', 'bus_stop_name'], axis = 1, inplace = True)

# Drop duplicates (in measurement or metric)
test.drop(['Tranc_YearMonth', 'Tranc_Month', 'floor_area_sqm', 'full_flat_type', 'lease_commence_date', 'year_completed', 'price_per_sqft'], axis = 1, inplace = True)

# Drop 'precinct_pavilion', because there's no adjacent feature that can support any analysis on this
test.drop(['precinct_pavilion'], axis = 1, inplace = True)

# Drop 'residential' and 'commercial', as we're assuming that it's all residential
test.drop(['residential', 'commercial'], axis = 1, inplace = True)

# Drop rental, we're not looking into rental, more of selling-buying
test.drop(['1room_rental', '2room_rental', '3room_rental', 'other_room_rental'], axis = 1, inplace = True)

# Drop irrelevant education/academic features
test.drop(['pri_sch_affiliation'], axis = 1, inplace = True)

test.head()

,id,town,flat_type,storey_range,flat_model,Tranc_Year,mid_storey,lower,upper,mid,...,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_name,vacancy,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation
0,114982,YISHUN,4 ROOM,07 TO 09,Simplified,2012,8,7,9,8,...,0,0,75.683952,426.467910,Ahmad Ibrahim Primary School,92,156.322353,Ahmad Ibrahim Secondary School,218,0
1,95653,JURONG WEST,5 ROOM,04 TO 06,Premium Apartment,2019,5,4,6,5,...,0,0,88.993058,439.756851,Jurong West Primary School,45,739.371688,Jurong West Secondary School,199,0
2,40303,ANG MO KIO,3 ROOM,07 TO 09,New Generation,2013,8,7,9,8,...,1,0,86.303575,355.882207,Jing Shan Primary School,36,305.071191,Anderson Secondary School,245,0
3,109506,WOODLANDS,4 ROOM,01 TO 03,New Generation,2017,2,1,3,2,...,0,0,108.459039,929.744711,Marsiling Primary School,54,433.454591,Woodlands Secondary School,188,0
4,100149,BUKIT BATOK,4 ROOM,16 TO 18,Model A,2016,17,16,18,17,...,0,0,113.645431,309.926934,Princess Elizabeth Primary School,40,217.295361,Bukit Batok Secondary School,223,0


#### Missing Values (NaN)

In [21]:
# find all columns with NaN (train)
nan_cols_train=train.columns[train.isna().any()].tolist()
# find number of rows with NaN for each column in nan_cols
for i in nan_cols_train:
    print(f'no. of NaN for {i}: {train[i].isna().sum()}')

print('\n')

# find all columns with NaN (test)
nan_cols_test=test.columns[test.isna().any()].tolist()
# find number of rows with NaN for each column in nan_cols
for i in nan_cols_test:
    print(f'no. of NaN for {i}: {test[i].isna().sum()}')

print('\n')

no. of NaN for Mall_Nearest_Distance: 829
no. of NaN for Mall_Within_500m: 92789
no. of NaN for Mall_Within_1km: 25426
no. of NaN for Mall_Within_2km: 1940
no. of NaN for Hawker_Within_500m: 97390
no. of NaN for Hawker_Within_1km: 60868
no. of NaN for Hawker_Within_2km: 29202


no. of NaN for market_hawker: 16737
no. of NaN for multistorey_carpark: 16737
no. of NaN for Mall_Nearest_Distance: 84
no. of NaN for Mall_Within_500m: 10292
no. of NaN for Mall_Within_1km: 2786
no. of NaN for Mall_Within_2km: 213
no. of NaN for Hawker_Within_500m: 10755
no. of NaN for Hawker_Within_1km: 6729
no. of NaN for Hawker_Within_2km: 3254




In [22]:
#check unique values for mall_nearest_distance
np.unique(train['Mall_Nearest_Distance'])

array([   0.        ,   34.26558116,   42.23320902, ..., 3471.145638  ,
       3496.40291   ,           nan])

In [23]:
#check relation between mall_nearest_distance to all mall-related features
train.loc[(train['Mall_Nearest_Distance'].isna()), ['Mall_Nearest_Distance','Mall_Within_500m','Mall_Within_1km','Mall_Within_2km']]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
75,NaN,NaN,NaN,NaN
321,NaN,NaN,NaN,NaN
478,NaN,NaN,NaN,NaN
643,NaN,NaN,NaN,NaN
691,NaN,NaN,NaN,NaN
...,...,...,...,...
150296,NaN,NaN,NaN,NaN
150388,NaN,NaN,NaN,NaN
150394,NaN,NaN,NaN,NaN
150479,NaN,NaN,NaN,NaN


In [24]:
#check if mall nearest distance NaN is because no mall within 2km
train.loc[(train['Mall_Nearest_Distance']>2000), ['Mall_Nearest_Distance','Mall_Within_500m','Mall_Within_1km','Mall_Within_2km']]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
109,2050.201309,NaN,NaN,NaN
220,2247.327448,NaN,NaN,NaN
662,2218.607002,NaN,NaN,NaN
672,2038.439070,NaN,NaN,NaN
798,2115.704584,NaN,NaN,NaN
...,...,...,...,...
149097,2103.873021,NaN,NaN,NaN
149446,2218.607002,NaN,NaN,NaN
149450,2260.225191,NaN,NaN,NaN
150095,2007.777071,NaN,NaN,NaN


*NaN value is not because there is no mall within 2km*

For all missing data in *mall_nearest_distance*, because kaggle requires submission of the same number of columns as given datasets, NaN value will be changed to 0. Since missing data is 0.5% of overall data, it will not have significant impact

Other data *mall_within_500m*, *mall_within_1km* , *mall_within_2km* *hawker_within_500m*, *hawker_within_1km*, *hawker_within_2km* NaN represents 0 count, so all NaN values will be changed to 0

##### Filling NaN values with 0

In [25]:
for i in nan_cols_train:
    train[i] = train[i].fillna(0)

for i in nan_cols_test:
    test[i] = test[i].fillna(0)


In [26]:
# check for cols with NaN (train)
def null_check(train):
    is_null = train.isna().sum().sort_values()
    is_null_df = is_null.to_frame()
    is_null_df = is_null_df.rename({0:'number of NaNs'}, axis = 1)
    is_null_df['number of rows'] = is_null_df['number of NaNs']/train.shape[0]
    is_null_df = is_null_df[is_null_df.loc[:]!=0].dropna()
    return is_null_df

null_check(train)

,number of NaNs,number of rows


In [27]:
# check for cols with NaN (test)
def null_check(test):
    is_null = test.isna().sum().sort_values()
    is_null_df = is_null.to_frame()
    is_null_df = is_null_df.rename({0:'number of NaNs'}, axis = 1)
    is_null_df['number of rows'] = is_null_df['number of NaNs']/train.shape[0]
    is_null_df = is_null_df[is_null_df.loc[:]!=0].dropna()
    return is_null_df

null_check(test)

,number of NaNs,number of rows


#### Renaming Columns
- Column names should be all lowercase.
- Column names should not contain spaces (underscores will suffice--this allows for using the `df.column_name` method to access columns in addition to `df['column_name']`).
- Column names should be unique and informative.

In [28]:
# Column name(s) change to Lowercase
train.columns= train.columns.str.lower()

# Column name(s) change to Lowercase
test.columns= test.columns.str.lower()

# Print to check
print(train.columns)
print(test.columns)

Index(['id', 'town', 'flat_type', 'storey_range', 'flat_model', 'resale_price',
       'tranc_year', 'mid_storey', 'lower', 'upper', 'mid', 'floor_area_sqft',
       'hdb_age', 'max_floor_lvl', 'market_hawker', 'multistorey_carpark',
       'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', 'planning_area', 'mall_nearest_distance',
       'mall_within_500m', 'mall_within_1km', 'mall_within_2km',
       'hawker_nearest_distance', 'hawker_within_500m', 'hawker_within_1km',
       'hawker_within_2km', 'hawker_food_stalls', 'hawker_market_stalls',
       'mrt_nearest_distance', 'mrt_name', 'bus_interchange',
       'mrt_interchange', 'bus_stop_nearest_distance',
       'pri_sch_nearest_distance', 'pri_sch_name', 'vacancy',
       'sec_sch_nearest_dist', 'sec_sch_name', 'cutoff_point', 'affiliation'],
      dtype='object')
Index(['id', 'town', 'flat_type', 'storey_range', 'flat_mo

## Save Cleaned Data

In [29]:
train.to_csv('datasets/train_clean.csv',index=False)
test.to_csv('datasets/test_clean.csv',index=False)